## User (ICP) Features Preprocessing

**Input:** Raw data tables (hidden for future research purposes)

**Output:** Array of ICP feature vectors in one-hot format

**Features used:**
* Unique user ID
* Industry code
* Text bytes (binned by quartile)
* Image bytes (binned by quartile)
* Video bytes (binned by quartile)

In [56]:
# Initial steps:
# Get list of dictionaries
# List index: ICP index
# List entry: Dictionary of ICP information
    # Industry
    # Num. text, image, video
    # Bytes text, image, video

# Only keep most recent data
    # 1. Sort dataframe by timestamp
    # 2. Iterate rows --> Keep row if current entry = empty

In [18]:
# How to compare timestamps
print "string" > "str"
print "20150507" > "20170629"

True
False


In [19]:
import pickle

interactions_pickle = '20170629-interactions-mappings.pkl'
with open(interactions_pickle, 'rb') as output:
    (interactions, iidx_to_cdn, cdn_to_iidx, uidx_to_icp, icp_to_uidx) = pickle.load(output)

In [20]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import csv

In [21]:
# Read in icpclassify file
icpclassify_filepath = 'icpstatistic/icpclassify.txt'
icpclassify_header = ['industry', 'icp']
icpclassify_datatypes = {
    'industry': str,
    'icp': str
}

icpclassify_df = pd.read_csv(icpclassify_filepath, 
                              sep=',', header=None, 
                              names=icpclassify_header,
                              dtype=icpclassify_datatypes)

In [22]:
icpclassify_df.head()

,industry,icp
0,0600,www.baidu.com
1,0100,www.qq.com
2,0300,www.taobao.com
3,0100,www.sina.com.cn
4,0500,www.weibo.com


In [23]:
print icpclassify_df.industry.unique()
print icpclassify_df.duplicated().unique()
print icpclassify_df.icp.unique().shape[0]

['0600' '0100' '0300' '0500' '0200' '0400']
[False]
2000


In [24]:
num_icps = len(icp_to_uidx.keys())
icp_list = icp_to_uidx.keys()

In [25]:
[x for x in icp_to_uidx.values() if icp_to_uidx.values().count(x) >= 2]
# No duplicate mappings. Just to check.

[]

In [26]:
# Next step: try something like this instead
icpclassify_dict = icpclassify_df.set_index('icp').T.to_dict()

In [27]:
print len(icpclassify_dict)

2000


In [28]:
icp_feature_dict_prelim = {icp:features for icp,features \
                    in icpclassify_dict.iteritems() if icp in icp_to_uidx}
print len(icp_feature_dict_prelim)
print icp_feature_dict_prelim["www.qq.com"] # Testing

1151
{'industry': '0100'}


In [29]:
# Check all ICPs in dict belong in RecSys
for icp, features in icp_feature_dict_prelim.iteritems():
    assert(icp in icp_to_uidx)

# Check all ICPs in RecSys matrix represented in feature dict
for icp in icp_to_uidx.keys():
    assert (icp in icp_feature_dict_prelim)

In [30]:
# For quick testing
def get_industry(icp):
    if icp in icp_list:
        return icp_feature_dict_prelim[icp]['industry']
    else:
        return 'ICP not in RecSys list'

In [31]:
get_industry('www.17ok.com')

'ICP not in RecSys list'

In [32]:
# TODO: Add num page elements, bytes data

# To sort this list based on mapping indices: 
# https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-values-of-the-dictionary-in-python

In [33]:
# Read in icpstatistic file
icpstatistic_filepath = 'icpstatistic/all_icp_statistics.txt'
icpstatistic_header = ['icp', 'textnum', 'imagenum', 'videonum', 'unknownnum',\
                       'textbytes', 'imagebytes', 'videobytes', 'unknownbytes',\
                       'createtime', 'ts']
icpstatistic_dtypes = {
    'icp': str,
    'textnum': np.int64,
    'imagenum': np.int64,
    'videonum': np.int64,
    'unknownnum': np.int64,
    'textbytes': np.int64,
    'imagebytes': np.int64,
    'videobytes': np.int64,
    'unknownbytes': np.int64,
    'createtime': str,
    'ts': str
}

icpstatistic_df = pd.read_csv(icpstatistic_filepath, 
                              sep=',', header=None, 
                              names=icpstatistic_header,
                              dtype=icpstatistic_dtypes)

In [34]:
icpstatistic_df.head()

,icp,textnum,imagenum,videonum,unknownnum,textbytes,imagebytes,videobytes,unknownbytes,createtime,ts
0,1000eb.com,52,1,0,0,437337,4286,0,0,20150312095107,20150102
1,2pcw.cn,2135,201,0,12,37115444,11604039,0,196756,20150312095108,20150102
2,360boclub.com,3312,93,7,9,143260320,2739491,78912,94623,20150312095108,20150102
3,365jia.cn,2607,5067,43,134,153051095,199486613,1402569193,7573289,20150312095108,20150102
4,365jilin.com,15,19,0,4,367434,255625,0,100506,20150312095108,20150102


In [35]:
# Sort: recent first --> only keep most recent data
icpstatistic_sorted = icpstatistic_df.sort_values(['ts', 'createtime'], ascending=[False, False])
icpstatistic_sorted.head()

,icp,textnum,imagenum,videonum,unknownnum,textbytes,imagebytes,videobytes,unknownbytes,createtime,ts
117957,1000eb.com,19364,5,0,0,370553410,21430,0,0,20150430003613,20150429
117958,2pcw.cn,4625,446,0,24,79126492,26706230,0,393512,20150430003613,20150429
117959,360boclub.com,176568,883,41,116,8153276319,32099895,464946,1928660,20150430003613,20150429
117960,365jia.cn,223157,199273,817,718,12508850846,8603710019,25164893261,903447074,20150430003613,20150429
117961,365jilin.com,78,55,0,5,1856159,224145,0,144808,20150430003613,20150429


In [36]:
icpstatistic_sorted.describe()

# Web data features: Bin by quartiles? 
# So: 0, 1st quartile (>0), 2nd quartile, 3rd, 4th

,textnum,imagenum,videonum,unknownnum,textbytes,imagebytes,videobytes,unknownbytes
count,1.421890e+05,1.421890e+05,142189.000000,1.421890e+05,1.421890e+05,1.421890e+05,1.421890e+05,1.421890e+05
mean,5.281273e+04,1.694368e+04,173.407289,1.685686e+03,1.802321e+09,8.872234e+08,1.070706e+09,8.917975e+08
std,1.850197e+05,1.858247e+05,3046.352977,1.612916e+04,7.427256e+09,9.001758e+09,2.863522e+10,1.779261e+10
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.541000e+03,5.900000e+01,0.000000,1.100000e+01,2.332635e+07,7.200290e+05,0.000000e+00,1.388590e+05
50%,1.978100e+04,1.016000e+03,0.000000,6.200000e+01,4.427659e+08,2.659960e+07,0.000000e+00,1.174268e+06
75%,7.217800e+04,1.014900e+04,6.000000,2.570000e+02,1.752729e+09,3.995624e+08,4.827420e+05,8.281184e+06
max,1.297565e+07,1.354639e+07,212220.000000,1.043031e+06,4.843496e+11,6.728541e+11,1.846369e+12,1.065438e+12


In [37]:
# Define quantiles for data feature bins
text_q1 = 0
text_q2 = icpstatistic_sorted['textbytes'].quantile(.25)
text_q3 = icpstatistic_sorted['textbytes'].quantile(.50)
text_q4 = icpstatistic_sorted['textbytes'].quantile(.75)

image_q1 = 0
image_q2 = icpstatistic_sorted['imagebytes'].quantile(.25)
image_q3 = icpstatistic_sorted['imagebytes'].quantile(.50)
image_q4 = icpstatistic_sorted['imagebytes'].quantile(.75)

video_q1 = 0
video_q2 = icpstatistic_sorted['videobytes'].quantile(.70)
video_q3 = icpstatistic_sorted['videobytes'].quantile(.80)
video_q4 = icpstatistic_sorted['videobytes'].quantile(.90)

In [38]:
icpstatistic_sorted['videobytes'].quantile(.54)

150.0

In [39]:
# Quantile-bin function:
def text_bin(bytes):
    if bytes <= text_q1: return 'q0'
    elif bytes > text_q1 and bytes < text_q2: return 'q1'
    elif bytes >= text_q2 and bytes < text_q3: return 'q2'
    elif bytes >= text_q3 and bytes < text_q4: return 'q3'
    elif bytes >= text_q4: return 'q4'
    else: return 'Error'
    
def image_bin(bytes):
    if bytes <= image_q1: return 'q0'
    elif bytes > image_q1 and bytes < image_q2: return 'q1'
    elif bytes >= image_q2 and bytes < image_q3: return 'q2'
    elif bytes >= image_q3 and bytes < image_q4: return 'q3'
    elif bytes >= image_q4: return 'q4'
    else: return 'Error'
    
def video_bin(bytes):
    if bytes <= video_q1: return 'q0'
    elif bytes > video_q1 and bytes < video_q2: return 'q1'
    elif bytes >= video_q2 and bytes < video_q3: return 'q2'
    elif bytes >= video_q3 and bytes < video_q4: return 'q3'
    elif bytes >= video_q4: return 'q4'
    else: return 'Error'

In [40]:
video_bin(21175721.0)

'q4'

In [41]:
# Add web content statistics to feature dict
for entry in icpstatistic_sorted.itertuples():
    icp = entry[1]
    
    # Hasn't used a CDN
    if icp not in icp_to_uidx: 
        continue
        
    # Has no entries yet
    if 'textnum' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['textnum'] = entry[2]
    if 'imagenum' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['imagenum'] = entry[3]
    if 'videonum' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['videonum'] = entry[4]
    if 'unknownnum' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['unknownnum'] = entry[5]
    if 'textbytes' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['textbytes'] = entry[6]
    if 'imagebytes' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['imagebytes'] = entry[7]
    if 'videobytes' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['videobytes'] = entry[8]
    if 'unknownbytes' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['unknownbytes'] = entry[9]
    if 'ts' not in icp_feature_dict_prelim[icp]:
        icp_feature_dict_prelim[icp]['ts'] = entry[11]

In [42]:
for icp, features in icp_feature_dict_prelim.iteritems():
    if 'industry' not in features: print icp
    if 'textnum' not in features: print icp
    if 'imagenum' not in features: print icp
    if 'videonum' not in features: print icp
    if 'unknownnum' not in features: print icp
    if 'textbytes' not in features: print icp
    if 'imagebytes' not in features: print icp
    if 'videobytes' not in features: print icp
    if 'unknownbytes' not in features: print icp

www.chinacourt.org
www.chinacourt.org
www.chinacourt.org
www.chinacourt.org
www.chinacourt.org
www.chinacourt.org
www.chinacourt.org
www.chinacourt.org


In [43]:
# Manually adding info for www.chinacourt.org - got listed as "chinacourt.org" in icpstatistic
# Please forgive me for this shitty code
for entry in icpstatistic_sorted.itertuples():
    icp_raw = entry[1]
    
    if 'chinacourt.org' in icp_raw:
        icp = 'www.chinacourt.org'
        
        if 'textnum' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['textnum'] = entry[2]
        if 'imagenum' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['imagenum'] = entry[3]
        if 'videonum' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['videonum'] = entry[4]
        if 'unknownnum' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['unknownnum'] = entry[5]
        if 'textbytes' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['textbytes'] = entry[6]
        if 'imagebytes' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['imagebytes'] = entry[7]
        if 'videobytes' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['videobytes'] = entry[8]
        if 'unknownbytes' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['unknownbytes'] = entry[9]
        if 'ts' not in icp_feature_dict_prelim[icp]:
            icp_feature_dict_prelim[icp]['ts'] = entry[11]
            
        break

In [44]:
for icp, features in icp_feature_dict_prelim.iteritems():
    if 'industry' not in features: print icp
    if 'textnum' not in features: print icp
    if 'imagenum' not in features: print icp
    if 'videonum' not in features: print icp
    if 'unknownnum' not in features: print icp
    if 'textbytes' not in features: print icp
    if 'imagebytes' not in features: print icp
    if 'videobytes' not in features: print icp
    if 'unknownbytes' not in features: print icp

In [45]:
icp_feature_dict_prelim['www.qq.com']

{'imagebytes': 51454,
 'imagenum': 40,
 'industry': '0100',
 'textbytes': 79575808,
 'textnum': 2716,
 'ts': '20150429',
 'unknownbytes': 3711144,
 'unknownnum': 160,
 'videobytes': 0,
 'videonum': 0}

In [46]:
# import pickle
# with open('20170703-icp-features-prelim.pkl', 'wb') as output:
#     pickle.dump(icp_feature_dict, output, -1)

In [47]:
icp_feature_dict_final = {
                          icp:dict(
                                icp=icp,
                                industry=features['industry'],
                                text_bin=text_bin(features['textbytes']),
                                image_bin=image_bin(features['imagebytes']),
                                video_bin=video_bin(features['videobytes'])
                          )
                          for icp, features in icp_feature_dict_prelim.iteritems()
}

In [48]:
# For testing purposes
video_heavy = [cdn for cdn, feature in icp_feature_dict_final.iteritems() if feature['video_bin'] == 'q4']
print len(video_heavy)
print video_heavy

154
['www.oppo.com', 'www.guolairen.com', 'www.nipic.com', 'www.dayoo.com', 'www.esteelauder.com.cn', 'www.hebei.com.cn', 'www.taisha.org', 'www.titan24.com', 'www.abbottmama.com.cn', 'www.icbc.com.cn', 'www.17173.com', 'www.go108.com.cn', 'www.313.com', 'www.cztv.com', 'www.9978.cn', 'nut.com.cn', 'www.nanhutravel.com', 'www.wifigx.com', 'www.dqdaily.com', 'www.cpic.com.cn', 'www.foodmate.net', 'www.ofweek.com', 'www.safehoo.com', 'www.55188.com', 'www.gxnews.com.cn', 'www.bmw.com.cn', 'www.sf-express.com', 'www.smzy.com', 'www.xa999.com', 'www.xiami.com', 'www.ce.cn', 'www.smartshe.com', 'www.gq.com.cn', 'www.koolearn.com', 'www.juooo.com', 'www.yesky.com', 'www.ffpic.com', 'www.feel-bar.com', 'www.kuwo.cn', 'www.sznews.com', 'www.pcgames.com.cn', 'www.voc.com.cn', 'www.hqew.com', 'www.cnnb.com.cn', 'www.0731fdc.com', 'www.xmhouse.com', 'www.6eat.com', 'www.xywy.com', 'www.chinanews.com', 'www.tingroom.com', 'www.weather.com.cn', 'www.zxart.cn', 'www.3dmgame.com', 'www.infinitus.com.

In [49]:
# Create a list of ICP feature dicts
# Ordered by uidx (user/ICP index)
icp_feature_list = [icp_feature_dict_final[uidx_to_icp[uidx]] for uidx in range(num_icps)]

In [50]:
icp_feature_list[0:5]

[{'icp': '365jilin.com',
  'image_bin': 'q1',
  'industry': '0100',
  'text_bin': 'q1',
  'video_bin': 'q0'},
 {'icp': 'www.100ec.cn',
  'image_bin': 'q4',
  'industry': '0100',
  'text_bin': 'q4',
  'video_bin': 'q2'},
 {'icp': 'www.163.com',
  'image_bin': 'q1',
  'industry': '0100',
  'text_bin': 'q2',
  'video_bin': 'q0'},
 {'icp': 'www.21cbh.com',
  'image_bin': 'q2',
  'industry': '0100',
  'text_bin': 'q4',
  'video_bin': 'q0'},
 {'icp': 'www.21cn.com',
  'image_bin': 'q3',
  'industry': '0100',
  'text_bin': 'q2',
  'video_bin': 'q2'}]

In [51]:
# Make sure indices match up
uidx_to_icp[1]

'www.100ec.cn'

In [52]:
# Vectorize! (One-hot encodings of each ICP)
from sklearn.feature_extraction import DictVectorizer
icp_vectorizer = DictVectorizer()
icp_feature_vectors = icp_vectorizer.fit_transform(icp_feature_list)

In [53]:
icp_feature_vectors.shape

(1151, 1172)

In [54]:
1172-1151
# 6 Industries
# 3 x 5 content-type bins

21

In [55]:
icp_feature_vectors

<1151x1172 sparse matrix of type '<type 'numpy.float64'>'
	with 5755 stored elements in Compressed Sparse Row format>

In [297]:
import pickle
with open('20170703-icp-feature-vectors.pkl', 'w') as output:
    pickle.dump(icp_feature_vectors, output, -1)